In [ ]:
#noetbook utilizado para a aula sobre sistemas de recomendação

#1) Carregar os dados
#2) Pré-processamento dos dados
#3) Divisão dos dados entre treinamento e teste
#4) Criar o sistema de recomendação através do ALS
#5) Realizar previsões
#6) Testar o modelo

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [5]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("app").getOrCreate() #cria a seção caso não exista ou obtém a já criada

# spark = SparkSession.builder.appName("recomendacao").config('spark.sql.execution.arrow.pyspark.enabled', True).config('spark.sql.session.timeZone', 'UTC').config('spark.driver.memory','32G').config('spark.ui.showConsoleProgress', True).config('spark.sql.repl.eagerEval.enabled', True).getOrCreate()



In [6]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
diretorioRecomendacao="/content/gdrive/MyDrive/Arquivos-MLib/Arquivos/cap5/u.data"  #diretório que contém o arquivo a ser utilizado

#1) Carregando o arquivo.

In [8]:
#lendo arquivos armazenados através da função genérica
rdd_movies = spark.sparkContext.textFile(diretorioRecomendacao)

In [9]:
rdd_movies.take(10)  #user id | item id | rating | timestamp

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [10]:
#definindo as bibliotecas a serem utilizadas
from pyspark.mllib.recommendation import ALS, Rating  #MLlib utilizada para implementar os algoritmos ALS e Rating

#2) Pré-processamento dos dados
#3) Separando os dados para treinamento e teste

In [11]:
#dividindo os dados entre treinamento e teste
(trainRatings, testRatings) = rdd_movies.randomSplit([0.7, 0.3])

In [12]:
trainRatings.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [13]:
testRatings.first()  #print da primeira linha do RDD

'298\t474\t4\t884182806'

In [14]:
trainingData = trainRatings.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))  #aplicando a função Rating

In [15]:
trainingData.first()  #print do RDD criado através da função Rating

Rating(user=196, product=242, rating=3.0)

In [16]:
#mesmo procedimento para os dados de teste
testData = testRatings.map(lambda l: l.split('\t')).map(lambda l: (int(l[0]), int(l[1])))

In [17]:
testData.first()  #id do usuário | id do filme

(298, 474)

#4) Construindo o modelo ALS

In [18]:
#definindo as variáveis do modelo
rank = 10  # número de fatores latentes do modelo  R->P (usuários)*Q (itens) => R_mxn = P_mxrank * Q_rankxm  (em que m= numero de usuário e n= numero de itens)
numIterations = 10 #número de iterações realizadas pelo modelo
model = ALS.train(trainingData, rank, numIterations, lambda_ = 0.02) # treina o modelo

#5) Previsão do modelo

In [19]:
model.predict(253, 465)  # entrada (usuário, produto)

4.084856720902144

In [20]:
previsao = model.predictAll(testData)  #previsão para todos os dados de teste
previsao.first()

Rating(user=178, product=454, rating=3.538299630892955)

In [21]:
previsao = previsao.map(lambda l: ((l[0], l[1]), l[2])) #mapea para a exibição
previsao.take(5)

[((178, 454), 3.538299630892955),
 ((648, 454), 3.09306860276592),
 ((201, 454), 2.346557136929568),
 ((145, 454), 4.0451656224319805),
 ((504, 1084), 3.3625073625056534)]

In [22]:
testRating2 = testRatings.map(lambda l: l.split('\t')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2]))) # mapea para exibição e utilização na análise

In [23]:
testRating2.first()

((298, 474), 4.0)

In [24]:
ratingsAndPredictions = testRating2.join(previsao)
ratingsAndPredictions.take(5)

[((210, 40), (3.0, 2.82986602796209)),
 ((234, 1184), (2.0, 1.1925795231410952)),
 ((225, 193), (4.0, 4.823144490020463)),
 ((249, 241), (5.0, 4.124041803919555)),
 ((178, 332), (3.0, 3.2814566683848008))]

#6) Avaliação do modelo

In [25]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Erro médio quadrado = " + str(MSE))

Erro médio quadrado = 1.1386946419822384


In [26]:
#dado um usuário -> recomenda os top 5 produtos
model.recommendProducts(105, 5) 

[Rating(user=105, product=921, rating=6.365865411141657),
 Rating(user=105, product=1160, rating=6.285028819979015),
 Rating(user=105, product=1204, rating=6.14284047608608),
 Rating(user=105, product=613, rating=6.074149045525904),
 Rating(user=105, product=1159, rating=6.017624278715991)]

In [27]:
#dado um produto -> recomenda top 5 usuários
model.recommendUsers(1, 5)  #filme Toy Story (1995)

[Rating(user=153, product=1, rating=5.785935930883823),
 Rating(user=732, product=1, rating=5.571304785860837),
 Rating(user=173, product=1, rating=5.504384506101772),
 Rating(user=939, product=1, rating=5.42164718024657),
 Rating(user=337, product=1, rating=5.299131275815698)]

In [28]:
#mostrando o vetor de características referentes usuários (coluna - P)
model.userFeatures().take(1)[0]

(2,
 array('d', [0.740386426448822, 1.0299545526504517, 0.6832942366600037, -0.2846042215824127, -0.3898579478263855, 0.5254072546958923, 0.7073284387588501, 0.8187930583953857, -0.3648652732372284, -0.748189389705658]))

In [29]:
#mostra o vetor de características referente a um produto (linha - Q)
model.productFeatures().take(1)[0]

(2,
 array('d', [0.07744257897138596, 0.05896100774407387, 0.089176245033741, -0.03062829189002514, -1.3208074569702148, 0.6180762052536011, 0.7630155086517334, 1.4909968376159668, -0.6862483620643616, 0.28639766573905945]))